# Robot Control

Kevin J. Walchko

created 20 Oct

-------------------

Now we are going to start getting the Roomba to move. In order to do that, we need to program the robot correctly. Students usually are lazy and write a lazy software flow for their robot and then don't understand why things crash or don't work like they want. Once we understand the Create 2 API and program a modular/logical software flow, we will develop a simple algorithm to get our robot to do obstacle avoidance.

## Objectives

- understand how to get the Roomba to move
- understand simple obstacle avoidance
- understand a simple mobile robot software flow

## References

- [pycreate2 python software](https://pypi.python.org/pypi/pycreate2)

## Setup

In [ ]:
%matplotlib inline
from __future__ import print_function
from __future__ import division
import numpy as np

# Making the Create Move

So we have already talked about mobile robots, coordinate systems, body frames, etc. Now we are going to talk about how do we command the robot and get it to go where we want.

```python
from  pycreate2 import Create2

# Create a Create2.
bot = Create2()

# Start the Create 2
bot.start()

# Put the Create2 into 'safe' mode so we can drive it
# This will still provide some protection
bot.safe()

# directly set the motor speeds ... go forward
bot.drive_direct(100, 100)  # inputs for motors are +/- 500 max
time.sleep(2)

# turn in place, CW
bot.drive_direct(200,-200)
time.sleep(2)
```

# Obsticle Avoidance

**insert flow chart here**

# Robot Software Architecture

There are many ways to setup how your robot's software should function. One extreme way is using the [Robot Operation System (ROS)](www.ros.org) which is not an operating system but rather an architecture for setting up a robot. On large systems, this is a good match, but there is a very steep learning curve and only runs on Ubuntu Linux.

- ROS pros
  - Free (BSD licensed) robotic architecture
  - Distributed architecture, supports both multi-process/core and multi-machine
    - Publish/subscribe architecture allows this flexibility
  - C++ and python interfaces
  - Broad academic adoption
- ROS cons
  - Rapid development cycle, leads to code working one year and broken due to new interfaces the next
    - Or you can just stay on an older version of ROS, but then you loose new advancements
  - Steep learning curve ... nothing is simple
  - Only can develop/run on Ubuntu Linux
  - Many packages written by Universities and are not always maintained to the current version or support is often lacking
  - Updated annually, but core packages do not always use the most current version for libraries
  - Complex build system with Catkin
  - Designed for relatively powerful computers and not optimized for small embedded devices like Raspberry Pi
  - No security, robots easily hacked
  - Reliant on `roscore` as the central pub/sub broker, moving large amounts of data around in messages (i.e., 3D lidar point clouds, large images, etc) can introduce unnecessary delays and CPU overhead
    - *Note:* this can be overcome by development of few nodes and smart partitioning of algorithms. However, this is not the default ROS mentality

This class will not throw you into the deep end with ROS. Instead we are just using Python.

## Suggested Simple Robot Architecture for Your Roomba

Actually you can write a rather professional, modular, and clean architecture with Python. Remember to always setup your system properly and tear it down when you shutdown. Killing your software with Ctrl-C (essentially causing it to crash) is sloppy and can leave you in a bad state.

```python
#!/usr/bin/env python

from __future__ import print_function, division

def Idel(robot):
    return True

def Autonomous(robot):
    # do things
    return True
```

```python
#!/usr/bin/env python

from __future__ import print_function, division
from modes import Autonomous   # this is a function you write in a file called modes.py
from pycreate2 import Create2  # Roomba driver
from nxp_imu import IMU        # imu driver
from time import sleep

class MyRobot(object):
    """
    This is a super simple class to hold things together.
    
    Unfortunately you are not trained like the rest of the world to
    program with classes, so I want to keep this simple. However, if
    you have any talent, then please try to do this properly.
    """
    bot = None
    camera = None
    imu = None
    
    def __init__(self, port):
        self.bot = Create2(port)
        self.camera = Camera('pi')
        self.camera.init(window=(640,480))
        
    def __del__(self):       
        self.bot.safe()
        self.bot.close()
        self.camera.close()
        self.imu.close()

if __name__ == "__main__":
    robot_modes = [Idle, Autonomous]
    port = "/dev/serial"  # serial port path, change as appropriate
    bot = MyRobot(port)
    current_mode = 0
    
    try:
        while True:
            robot_modes[current_mode](bot)
            sleep(0.1)
    except KeyboardInterrupt:
        print('User hit Ctrl-C, robot shutting down\n\nBye ...')
                
```